# Анализ данных об ожидаемой продолжительности жизни в субъектах РФ

## Цель исследования

В качестве цели исследования поставим анализ и визуализацию данных об ожидаемой продолжительности жизни в РФ и определение наличия или отсутствия статистической зависимости этого параметра от численности населения на одного врача.

## Данные для исследования

Для выполнения данной работы воспользуемся данными Федеральной службы государственной статистики (Росстат). Будем использовать следующие показатели по каждому субъекту федерации:
- Ожидаемая продолжительность жизни при рождении за 1990-2023 годы (`data/rosstat_life_expectancy.xlsx`)
- Численность населения на одного врача за 1997-2017 годы (`data/rosstat_population_per_doctor.xlsx`)

Также используем систематизированные данные о субъектах РФ из [GitHub-репозитория](https://github.com/arbaev/russia-cities). Данные представлены в формате JSON в файле `./data/russia-regions.json`.

Для построения карты будем использовать данные о границах субъектов из [GitHub-репозитория](https://github.com/timurkanaz/Russia_geojson_OSM) (файл `data/Russia_regions.geojson`)

## Загрузка данных

Для анализа данных будем использовать язык Python и библиотеку `pandas`. Загрузим данные об ожидаемой продолжительности жизни в `DataFrame`.

In [ ]:
import pandas as pd

DF_LIFE_EXPECTANCY = pd.read_excel('./data/rosstat_life_expectancy.xlsx', sheet_name='Отчет', header=1)[2:]
DF_LIFE_EXPECTANCY.head()


Загрузим данные (ОКАТО и Федеральный округ) о 83 субъектах РФ в `DataFrame` `DF_REGIONS`.

In [ ]:
DF_REGIONS = pd.read_json('./data/russia-regions.json')[['okato', 'district']]
DF_REGIONS.head(8)

Напишем функцию для фильтрации `DataFrame`, оставляющую только данные о 83 субъектах из `DF_REGIONS`.

In [ ]:
def filter_regions(target_df: pd.DataFrame, regions_df: pd.DataFrame) -> pd.DataFrame:
    return target_df[target_df['ОКАТО'].isin(regions_df['okato'])]

Исправим названия колонок в `DF_LIFE_EXPECTANCY`: Добавим названия для первых двух (Регион и ОКАТО - Общероссийский Классификатор Объектов Административно-Территориальных Образований), остальные (года измерений) приведём к типу `int`.

Заменим нулевые значения на `numpy.nan`.

Отфильтруем субъекты с помощью ранее написанной функции `filter_regions`.

Также исправим индексацию (на `0, 1, ..., n`) с помощью метода `DataFrame.reset_index`.

In [ ]:
import numpy as np

def transform_years(year: str) -> int | str:
    if len(year.split()) == 1:
        return year
    return int(year.split()[0])


new_names = {
    "Unnamed: 0": "Регион",
    "Unnamed: 1": "ОКАТО"
}

DF_LIFE_EXPECTANCY.rename(new_names, axis='columns', inplace=True)
DF_LIFE_EXPECTANCY.rename(transform_years, axis='columns', inplace=True)

DF_LIFE_EXPECTANCY['ОКАТО'] = DF_LIFE_EXPECTANCY['ОКАТО'].astype(int)

DF_LIFE_EXPECTANCY.replace(0, np.nan, inplace=True)

DF_LIFE_EXPECTANCY = filter_regions(DF_LIFE_EXPECTANCY, DF_REGIONS)

DF_LIFE_EXPECTANCY.reset_index(drop=True, inplace=True)

DF_LIFE_EXPECTANCY

Напишем функцию для сокращения названий некоторых субъектов `replace_long_names` (в целях улучшения читаемости), она принимает на вход `DataFrame` и меняет в нём все заданные названия на более короткие.

Применим эту функцию к `DF_LIFE_EXPECTANCY`.

In [ ]:
def replace_long_names(df: pd.DataFrame) -> None:
    names_to_replace = {
        "Город Москва столица Российской Федерации город федерального значения": "Город Москва",
        "Город федерального значения Севастополь": "Город Севастопль",
        "Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)": "Тюменская область (кроме ХМАО и ЯНАО)"
    }

    for old_name, new_name in names_to_replace.items():
        df.replace(old_name, new_name, inplace=True)
replace_long_names(DF_LIFE_EXPECTANCY)

In [ ]:
DF_LIFE_EXPECTANCY.describe()

Аналогичные действия выполним для численности населения на одного врача.

In [ ]:
DF_POPULATION_PER_DOCTOR = pd.read_excel('./data/rosstat_population_per_doctor.xlsx', sheet_name='Отчет', header=1)

DF_POPULATION_PER_DOCTOR.rename(new_names, axis='columns', inplace=True)
DF_POPULATION_PER_DOCTOR.rename(transform_years, axis='columns', inplace=True)

DF_POPULATION_PER_DOCTOR.replace(0, np.nan, inplace=True)

DF_POPULATION_PER_DOCTOR = filter_regions(DF_POPULATION_PER_DOCTOR, DF_REGIONS)

replace_long_names(DF_POPULATION_PER_DOCTOR)

DF_POPULATION_PER_DOCTOR.reset_index(drop=True, inplace=True)

DF_POPULATION_PER_DOCTOR.head()

In [ ]:
DF_POPULATION_PER_DOCTOR.describe()

Итак, после загрузки и первичной обработки данных Росстата мы получили 2 `DataFrame'a`:
- `DF_LIFE_EXPECTANCY` - основной набор данных, содержит данные об ожидаемой продолжительности жизни при рождении по субъектам РФ за 1990-2023 годы
- `DF_POPULATION_PER_DOCTOR` - данные о численности населения на одного врача по субъектам РФ за 1997-2017 годы

Также получен `DataFrame` `DF_REGIONS` с данными о регионах РФ.

Ожидаемую прододжительность жизни (ОПЖ) при рождении Росстат вычисляет по следующей формуле:
$$e(0)=\frac{T_0}{I_0},$$
где $T_0$ - число человеко-лет, которое предстоить прожить гипотетической когорте в 100 000 человек (на основании возраста текущего населения);
$I_0$ - численность гипотетической когорты (для ОПЖ при рождении - 100 000 человек).

## Анализ ожидаемой продолжительности жизни

Проанализируем `DF_LIFE_EXPECTANCY`.

Посчитаем среднюю продолжительность жизни для регионов, в которых проводились измерения во все годы в 1990-2023, сохраним в `pandas.Series` и построим график с 5 субъектами, где показатель наивысший, и с 5 субъектами, где показатель наименьший.

In [ ]:
mean_life_expectancy = DF_LIFE_EXPECTANCY.dropna().drop("ОКАТО", axis=1).set_index("Регион").mean(axis=1).sort_values()
mean_life_expectancy

Графики будем строить с помощью библиотеки `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt

plt.figure("1", (8, 4))

plt.barh(mean_life_expectancy[:5].index, mean_life_expectancy[:5], align='center', color='red')
plt.barh(mean_life_expectancy[-5:].index, mean_life_expectancy[-5:], align='center', color='green')

plt.grid()
plt.gca().xaxis.tick_top()
plt.gca().invert_yaxis()
plt.xticks(range(0, 76, 5))
plt.title("Средняя за 1990-2023 гг. ожидаемая продолжительность жизни, лет")

plt.tight_layout()
plt.savefig('./images/life_expectancy_top.png', transparent=True)

plt.show()

Получаем, что самая низкая средняя ожидаемая продолжительность жизни (ОПЖ) при рождении за рассматриваемый период наблюдалась в Республике Тыва, а самая высокая -  в Республике Дагестан.

Найдём среднее значение ожидаемой продолжительности жизни по каждому году среди всех регионов и сохраним в `Series`.

In [ ]:
mean_life_expectancy_by_years = DF_LIFE_EXPECTANCY.T[2:].mean(axis=1)
mean_life_expectancy_by_years.head()

Построим график ожидаемой продолжительности жизни за 1990-2023 годы. Нанесем найденные ранее субъекты с максимальными и минимальными значениями. Добавим средние значения из `mean_life_expectancy_by_years` и средние значения по группам максимальных и минимальных регионов.

In [ ]:
plt.figure("2", (20, 10))


plt.plot(mean_life_expectancy_by_years, '-o', label="Среднее по всем регионам")

for i, region in enumerate(mean_life_expectancy.index[:5]):
    plt.plot(DF_LIFE_EXPECTANCY[DF_LIFE_EXPECTANCY['Регион'] == region].iloc[0][2:], label='Группа с min ОПЖ' if i == 0 else '', color='#fca89f')


min_regions = DF_LIFE_EXPECTANCY[DF_LIFE_EXPECTANCY['Регион'].isin(mean_life_expectancy.index[:5])]
mean_life_expectancy_min = min_regions.T[2:].mean(axis=1)

plt.plot(mean_life_expectancy_min, '-o', label='Среднее среди минимальных', color='#f80000')

for i, region in enumerate(mean_life_expectancy.index[-5:]):
    plt.plot(DF_LIFE_EXPECTANCY[DF_LIFE_EXPECTANCY['Регион'] == region].iloc[0][2:], label='Группа с max ОПЖ' if i == 0 else '', color='#badbad')

max_regions = DF_LIFE_EXPECTANCY[DF_LIFE_EXPECTANCY['Регион'].isin(mean_life_expectancy.index[-5:])]
mean_life_expectancy_max = max_regions.T[2:].mean(axis=1)

plt.plot(mean_life_expectancy_max, '-o', label='Среднее среди максимальных', color='#006600')

plt.xticks(range(1990, 2024))
plt.yticks(range(53, 81))
plt.margins(x=0.01)
plt.legend()
plt.grid()
plt.xlabel("Год")
plt.ylabel("Ожидаемая продолжительность жизни")
plt.title("График ОПЖ при рождении за 1990-2023 годы")

plt.tight_layout()
plt.savefig('./images/regions_expectancy_plot.png', transparent=True)

plt.show()

По графику видим, что между средними "минимальной" и "максимальной" группами есть существенный отрыв (около 5-10 лет). Построим диаграмму для иллюстрации этого отрыва.

In [ ]:
from matplotlib.colors import Normalize

plt.figure("3", (18, 7))

difference_mean = mean_life_expectancy_max - mean_life_expectancy_min

normalizer = Normalize(difference_mean.min(), difference_mean.max())
colors = plt.cm.RdYlGn_r(normalizer(list(difference_mean)))

plt.bar(difference_mean.index, difference_mean, color=colors)

plt.xticks(range(1990, 2024))
plt.yticks(range(0, 13))
plt.grid(axis='y')
plt.margins(x=0.01)

plt.xlabel('Год')
plt.ylabel('Разница в ОПЖ, лет')
plt.title('Разрыв между средними ОПЖ "минимальной" и "максимальной" групп')

plt.tight_layout()
plt.savefig('./images/regions_expectancy_difference.png', transparent=True)

plt.show()

Видим, что до 2000 года разрыв был около 7 лет, затем произошел серьезный скачок, и до 2013 года разрыв между средними в этих группах составлял около 11 лет. Далее началось сокращение этого разрыва (2014-2021 гг.). Но в последние годы (2021-2023) разница между этими группами вновь стала расти.

Посмотрим на динамику ОПЖ при рождении в различных федеральных округах РФ. Для этого соберём средние данные по ним в `DataFrame` `mean_by_districts`.

In [ ]:
mean_by_districts = DF_LIFE_EXPECTANCY.merge(DF_REGIONS, left_on='ОКАТО', right_on='okato')\
                                        .drop(['okato', 'ОКАТО', 'Регион'], axis='columns')\
                                        .groupby('district').mean()
mean_by_districts

По полученным данным построим график.

In [ ]:
plt.figure("4", (20, 10))

plt.plot(mean_by_districts.T, '-o', label=mean_by_districts.index)
plt.plot(mean_life_expectancy_by_years, label='Среднее по всем регионам', linewidth=7)

plt.xticks(range(1990, 2024))
plt.yticks(range(60,79))
plt.margins(x=0.01)
plt.legend(fontsize=15)
plt.grid()
plt.xlabel("Год")
plt.ylabel("Ожидаемая продолжительность жизни")
plt.title("График ОПЖ при рождении за 1990-2023 годы (федеральные округа)")

plt.tight_layout()
plt.savefig('./images/districts_expectancy_plot.png', transparent=True)

plt.show()

По графику видим, что самые высокие показатели (и значительно выше среднего по регионам) стабильно демонстрирует Северо-Кавказский федеральный округ. Самые низкие значения наблюдаются в Дальневосточном федеральном округе.

Построим диаграмму, отражающую среднее отклонение для каждого ФО от среднего значения по стране.
Значения среднего отклонения поместим в `Series` `mean_deviation_by_districts`.

In [ ]:
mean_deviation_by_districts = (mean_by_districts - mean_life_expectancy_by_years).mean(axis=1)
mean_deviation_by_districts

In [ ]:
colors = ['#f13a13' if value < 0 else '#2bb52b' for value in mean_deviation_by_districts]

plt.figure("5", (13, 6))
plt.bar(mean_deviation_by_districts.index, mean_deviation_by_districts, color=colors)

plt.grid(axis='y')
plt.margins(x=0.02)
plt.yticks(np.arange(-3.0, 4.6, 0.5))
plt.xlabel('Федеральный округ')
plt.ylabel('Среднее отклонение, лет')
plt.title('Среднее отклонение ОПЖ в ФО от среднего по всем регионам')

plt.tight_layout()
plt.savefig('./images/districts_mean_deviation.png', transparent=True)

plt.show()

По диаграмме видим, что в Северо-Кавказском ФО в среднем ОПЖ больше на 4,5 года среднего по стране, а в Дальневосточном ФО - меньше на почти 3 года.

Постром диаграмму `boxplot` по средним значениям регионов за рассматриваемый период по федеральным округам, чтобы определить рассеяность значений внутри каждого ФО.

In [ ]:
districts = DF_REGIONS['district'].unique()
district_data = dict()
for district in districts:
    district_okatos = DF_REGIONS[DF_REGIONS['district'] == district]['okato']
    district_df = DF_LIFE_EXPECTANCY[DF_LIFE_EXPECTANCY['ОКАТО'].isin(district_okatos)].drop(['ОКАТО', 'Регион'], axis='columns')
    district_data[district] = list(district_df.mean(axis=1))
district_data

In [ ]:
import seaborn as sns

plt.figure("6", (12, 5))
sns.boxplot(district_data)

plt.yticks(range(60, 78))
plt.xlabel('Федеральный округ')
plt.ylabel('ОПЖ при рождении, лет')
plt.grid(axis='y')
plt.title('Распределение ОПЖ при рождении в федеральных округах')

plt.tight_layout()
plt.savefig('./images/districts_boxplot.png', transparent=True)

plt.show()

In [ ]:
mean_life_expectancy['Город Москва']

По диаграмме видим, что наиболее рассеяны значения в Сибирском ФО, а меньше всего ОПЖ варьируется в Южном ФО. Также данная диаграмма вновь иллюстрирует превосходство Северо-Кавказского ФО по ОПЖ над другими ФО. В Центральном ФО наблюдается выброс (примерно 72.5 года), это среднее значение ОПЖ в Москве.

Построим таблицу с параметрами ОПЖ по федеральным округам.

In [ ]:
table_data = DF_LIFE_EXPECTANCY.merge(DF_REGIONS, left_on='ОКАТО', right_on='okato')\
                                        .drop(['okato', 'ОКАТО', 'Регион'], axis='columns')
table = pd.DataFrame(index=DF_REGIONS['district'].unique(), columns=['min 1990', 'mean 1990', 'max 1990',
                                                                     'min 2023', 'mean 2023', 'max 2023'])
table.index.name = 'Федеральный округ'

table['min 1990'] = table_data.groupby('district')[1990].min()
table['mean 1990'] = table_data.groupby('district')[1990].mean().round(2)
table['max 1990'] = table_data.groupby('district')[1990].max()

table['min 2023'] = table_data.groupby('district')[2023].min()
table['mean 2023'] = table_data.groupby('district')[2023].mean().round(2)
table['max 2023'] = table_data.groupby('district')[2023].max()

table['mean difference'] = table['mean 2023'] - table['mean 1990']
table.loc['Среднее'] = table.mean(axis=0)
table

Видим, что во всех федеральных округах средние значения за рассматриваемый период выросли (в среднем 3.70 года). Наибольший прирост наблюдался в Северо-Кавказском ФО, наименьший - в Дальневосточном ФО.

Выведем карту с 83 субъектами РФ для иллюстрации средней ОПЖ при рождении за 1990-2023 годы. Для этого воспользуемся файлом GeoJSON по субъектам РФ (`data/Russia_regions.geojson`). Также будем использовать бибилиотеки `GeoPandas` для работы с географическими данными (тип `geometry`) и `folium` для отображения данных на карте Open Street Map.

In [ ]:
import geopandas as gpd

GEO_DF = gpd.read_file('./data/Russia_regions.geojson')
GEO_DF

Сопоставим названия регионов с данными из `russia-regions.json`, чтобы получить номера ОКАТО субъектов.

In [ ]:
reg_df = pd.read_json('./data/russia-regions.json')
reg_df['fullname'] = reg_df['fullname'].str.casefold()
reg_df['fullname'] = reg_df['fullname'].str.replace('автономная область', 'ао').str.replace('автономный округ', 'ао')
reg_df['fullname'] = reg_df['fullname'].str.replace('область', 'обл.')
reg_df['fullname'] = reg_df['fullname'].str.replace(r' ?/[ а-я]*/?', '', regex=True)

GEO_DF['region'] = GEO_DF['region'].str.casefold()
GEO_DF['region'] = GEO_DF['region'].str.replace(r'\([а-я]*\)', '', regex=True)
GEO_DF['region'] = GEO_DF['region'].str.replace('г. ', '')
GEO_DF['region'] = GEO_DF['region'].str.replace(' - чувашия', '')
GEO_DF = GEO_DF[GEO_DF['region'].isin(['республика крым', 'город федерального значения севастополь']) == False]
GEO_DF['region'] = GEO_DF['region'].apply(lambda s: s.strip())
GEO_DF = GEO_DF.merge(reg_df, left_on='region', right_on='fullname')[['okato', 'geometry']]
GEO_DF.set_index('okato', inplace=True)

GEO_DF

Добавим в `GeoDataFrame` `GEO_DF` данные о средней ОПЖ при рождении за 1990-2023 годы.

In [ ]:
mean_by_okato = DF_LIFE_EXPECTANCY.drop('Регион', axis='columns').set_index('ОКАТО').mean(axis=1)

GEO_DF['mean'] = mean_by_okato
GEO_DF['name'] = DF_LIFE_EXPECTANCY.set_index('ОКАТО')['Регион']
GEO_DF['district'] = DF_REGIONS.set_index('okato')['district']
GEO_DF.reset_index(inplace=True)
GEO_DF

Нанесём полученные значения на карту.

In [ ]:
import folium

mean_map = folium.Map(location=[62.800867, 101.903995], zoom_start=3)

tooltip = folium.GeoJsonTooltip(
    fields=["name", "okato", "district", "mean"],
    aliases=["Регион", "ОКАТО", "Федеральный округ", "Средняя ОПЖ, лет"],
    sticky=True,
    labels=True
)

folium.Choropleth(
    geo_data=GEO_DF,
    name="choropleth",
    data=GEO_DF,
    columns=["okato", "mean"],
    key_on="feature.properties.okato",
    fill_color="RdYlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Средняя ОПЖ при рождении за 1990-2023 гг., лет"
).add_to(mean_map)

def get_style(feature):
    return {
        "fillOpacity": 0,
        "color": "balck"
    }

folium.GeoJson(GEO_DF, tooltip=tooltip, style_function=get_style).add_to(mean_map)

mean_map.save('./maps/mean_map.html')
mean_map

Полученная карта наглядно демонстрирует отставание Дальневостчного и Сибирского ФО и лидерство Центрального, Южного и Северо-Кавказского ФО.

Отображим карту с возможностью узнать ОПЖ при рождении за каждый 1990-2023 годы (добавим слайдер с помощью плагина `TimeSliderChoropleth`).

In [ ]:
from folium.plugins import TimeSliderChoropleth
import branca.colormap as cm


years_map_data = pd.merge(GEO_DF, DF_LIFE_EXPECTANCY, left_on='okato', right_on='ОКАТО')[['okato', 'name', 'geometry'] + list(DF_LIFE_EXPECTANCY.columns[2:])]

min_val = DF_LIFE_EXPECTANCY[DF_LIFE_EXPECTANCY.columns[2:]].min().min()
max_val = DF_LIFE_EXPECTANCY[DF_LIFE_EXPECTANCY.columns[2:]].max().max()

datetime_index = (pd.date_range("1990","2024", freq='YE').astype("int64") // 10 ** 9).astype("U10")
datetime_index

styledata = {}
cmap = cm.linear.RdYlGn_03.scale(min_val, max_val)

for okato in years_map_data['okato']:
    df = pd.DataFrame(
        {
            "color": [(cmap(years_map_data[years_map_data['okato'] == okato].iloc[0][year]) if not (np.isnan(years_map_data[years_map_data['okato'] == okato].iloc[0][year])) else '#e3e8e6') for year in range(1990, 2024)],
            "opacity": 1
        },
        index=datetime_index
    )
    styledata[okato] = df

styledict = {
    str(okato): data.to_dict(orient="index") for okato, data in styledata.items()
}

years_map = folium.Map(location=[62.800867, 101.903995], zoom_start=3)

TimeSliderChoropleth(
    years_map_data.set_index('okato').to_json(),
    styledict=styledict,
).add_to(years_map)

years_map.save('./maps/years_map.html')
years_map

Видно, что после 2019 года (в 2020 и 2021 годах) карта заметно бледнеет (т. е. ОПЖ при рождении в регионах уменьшается), что, вероятно, связанно с пандемией COVID-19. Построим таблицу по ФО со средними значениями в 2019, 2020, 2021 и 2022 годах.

In [ ]:
years = range(2019, 2023)
table = pd.DataFrame(index=DF_REGIONS['district'].unique(), columns=[f"mean {year}" for year in years])
table.index.name = 'Федеральный округ'

for year in years:
    table[f"mean {year}"] = table_data.groupby('district')[year].mean()

table.loc['Среднее'] = table.mean(axis=0)

table

Видим численное уменьшение средних значений ОПЖ при рождении в 2020 и 2021 годах. Построим график для иллюстрации данного явления.

In [ ]:
plt.figure("7", (13, 6))

plt.plot(table.rename(lambda name: name.split()[1], axis='columns').T, '-o', label=table.index)
plt.grid()
plt.legend(loc=1)
plt.margins(x=0.01)
plt.yticks(range(67, 78))
plt.xlabel('Год наблюдений')
plt.ylabel('Средняя ОПЖ при рождении в ФО')
plt.title('Падение средних значений ОПЖ при рождении в 2020-2021 гг.')

plt.tight_layout()
plt.savefig('./images/covid_decrease.png', transparent=True)

plt.show()

## Проверка зависимости ОПЖ при рождении и населения на 1 врача

Проверим наличие статистической зависимости ОПЖ при рождении с количеством населения на одного врача. При вычислении ОПЖ при рождении напрямую не используется параметр населения на 1 врача и наоброт, поэтому такая проверка не лишена смысла.

Построим график, на котором отобразим средние значение этих двух параметров по всем регионам с 1997 по 2017 годы.

In [ ]:
fig, ax1 = plt.subplots()

fig.set_size_inches(10, 5)

ax1.plot(DF_LIFE_EXPECTANCY[range(1997, 2018)].mean(axis=0), '-o', color='green')
ax1.tick_params(axis='y', colors='green')
ax1.set_ylabel('Средняя по стране ОПЖ при рождении, лет')

plt.xticks(range(1997, 2018))
plt.grid()
plt.margins(x=0.02)

ax2 = ax1.twinx()
ax2.plot(DF_POPULATION_PER_DOCTOR[range(1997, 2018)].mean(axis=0), '-o', color='red')
ax2.tick_params(axis='y', colors='red')
ax2.set_ylabel('Среднее по регионам население на 1 врача, чел.')

plt.tight_layout()
plt.title('Связь между населением на 1 врача и ОПЖ при рождении')

plt.tight_layout()
plt.savefig('./images/medicine_relation_plot.png', transparent=True)

plt.show()

Видим, что есть тенденция увелечения средней ОПЖ при рождении при уменьшении населения на 1 врача. Построим диагрмму разброса (scatter-plot) для этих параметров.

In [ ]:
plt.figure("8")
plt.scatter(DF_LIFE_EXPECTANCY[range(1997, 2018)].mean(axis=0), DF_POPULATION_PER_DOCTOR[range(1997, 2018)].mean(axis=0))
plt.grid()
plt.xlabel('Средняя ОПЖ при рождении, лет')
plt.ylabel('Среднее население на 1 врача, чел.')
plt.title('Связь между ОПЖ и населением на 1 врача')

plt.tight_layout()
plt.savefig('./images/medicine_relation_scatter.png', transparent=True)

plt.show()

Эта диаграмма так же демонстрирует потенциальную отрицательную корреляцию между двумя рассматриваемыми параметрами. С помощью функции `scipy.stats.pearsonr` оценим корреляцию между этими величинами.

In [ ]:
from scipy.stats import pearsonr

result = pearsonr(DF_LIFE_EXPECTANCY[range(1997, 2018)].mean(axis=0), DF_POPULATION_PER_DOCTOR[range(1997, 2018)].mean(axis=0))
result.statistic

Полученное значение (примерно `-0.625`) свидетельствует о наличии значительной отрицательной корреляции и статистической взаимосвязи между ОПЖ и населением на 1 врача, что, однако, не говорит о наличии причинно-следственной связи, но, как минимум, ставит этот вопрос для дальнейших исследований.

## Вывод

В результате выполнения данной работы был проведен анализ данных об ожидаемой продолжительности жизни, выявлены субъекты с минимальными и максимальными показателями, проанализирован разрыв между ними. Также был проанализирован показатель ОПЖ при рождении для федеральных округов РФ. Для визуализации данных и выявления тенденций были построены графики, диаграммы и карты. Была выявлена значительная отрицательная корреляция между средним по стране ОПЖ при рождении и средним по стране населением на 1 врача, что ставит вопрос о наличии причинно-следственной связи между этими показателями.